### Homework 13 <br> Zain Hassan <br> zh06624@st.habib.edu.pk

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk import pos_tag, word_tokenize, RegexpParser
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from bs4 import BeautifulSoup
import requests
import re
import numpy
import tweepy as tw
import json
import pandas as pd
import numpy as np
from textblob import TextBlob

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

#### Q1: Select any article on the website of your interest and perform Name Entity Relation (NER) analysis on it. Show all the necessary output and visualization and explain your findings. Also mention if you see any shortcoming or error in the algorithm output.

In [ ]:
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    for script in soup(["script", "style", 'aside']):
        script.extract()
        return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
ny_bb = url_to_string('https://www.nba.com/news/kings-hope-for-crowd-to-carry-them-in-game-7-vs-warriors')
article = nlp(ny_bb)
len(article.ents)

In [ ]:
sentences = [x for x in article.sents]
print(sentences[0])

In [ ]:
displacy.render(nlp(str(sentences)), jupyter=True, style='ent')

##### We can the whole article is tagged with name entity recognization and it had marked different entities such as person, location, organization, events and date.

In [ ]:
labels = [x.label_ for x in article.ents]
Counter(labels)

##### We can see the frequency of each entity found from our article

In [ ]:
items = [x.text for x in article.ents]
Counter(items).most_common(5)

##### We found out the 5 most commom entities in the article

#### Errors in the algorithm

TVNewsHomeTop StoriesAwardsFeaturesDraftHistoryLearn -> PERSON - This is not a person<br>
PassCustomer Support -> PERSON - This is not a person<br>
Game 5 -> LAW - This ia an event
Butler -> ORG - This is a person

#### Q2: Select any research paper of your interest and perform word frequency and keyword extraction analysis. Use at least 3 different techniques/libraries. Show all the necessary output and visualization and explain your findings. Compare different techniques and mention if you see any shortcoming or error in the algorithm output.

In [ ]:

from tika import parser
raw= parser.from_file('research.pdf')

In [ ]:
tokenize_words = nltk.tokenize.word_tokenize(raw['content'])
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(["T", "J", "D", "S", "M", "H", "et al", "et", "pp", "a.", "m.", "d.", "j.", "s.", "r.", "c.", "t."])
no_stopwords = [word for word in tokenize_words if word.lower() not in stopwords]

# remove punctuation, count raw words
nonPunct = re.compile('.*[A-Za-z].*')
clean_words = [w for w in no_stopwords if nonPunct.match(w)]
    
filtered_word_freq = nltk.FreqDist(clean_words)

In [ ]:
filtered_word_freq.most_common(20)

In [ ]:
filtered_word_freq.plot(30,title='Frequency distribution for 30 most common tokens in our text collection (excluding stopwords and punctuation)')

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]


In [ ]:
path='07558_CenturyGothic.ttf'
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white", font_path=path).generate(raw['content'])
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
stopwords = set(STOPWORDS)
stopwords.update(["T", "J", "D", "S", "M", "H", "et al", "et", "al"])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, max_font_size=50, max_words=100, background_color="white").generate(raw['content'])

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
from rake_nltk import Rake
r = Rake()
r.extract_keywords_from_text(raw['content'])
r.get_ranked_phrases_with_scores()[:10]

In [ ]:
nlp = spacy.load('en_core_web_sm')
doc2 = nlp(raw['content'])
print(doc2.ents)

In [ ]:
import yake
kw_extractor = yake.KeywordExtractor()
language = "en"
max_ngram_size = 2
deduplication_threshold = 0.9
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(raw['content'])
for kw in keywords:
    print(kw)

In [ ]:
# # importing required modules
# from PyPDF2 import PdfReader

# # creating a pdf reader object
# reader = PdfReader('research.pdf')

# # printing number of pages in pdf file
# print(len(reader.pages))

# # getting a specific page from the pdf file
# page = reader.pages[0]

# # extracting text from page
# raw = page.extract_text()
# # print(text)


### Q3: 3. Choose any keyword of your interest and pull at least 1000 (tweets) from Twitter.
#### a. Clean the tweets and store them in csv file.


In [ ]:
consumer_key = 'eR1zb853CsroHzovzLiJNPBK8'
consumer_key_secret = 'SMTGv6dLoAj81HAJ1q0I8DQ19pTTscdAINCXynLHdxb6tUDVjA'
access_token = '107436862-6ywCOzDd5bSHLZd3BWJ3buacPsL4OpWmhV5Lu9Uk'
access_token_secret = 'JAfDMXeg0OYz9ax8dELqvSlfX1sduHDVHnslVWwJjiJc2'
auth = tw.OAuth2BearerHandler('AAAAAAAAAAAAAAAAAAAAAM6CnAEAAAAAg3jcrw7guJg1MWNZ6avdp1hP2sk%3DMLBlokDJlF6H7yNGQ5OrS5WWT0Yqb16rvADj9pvYXLhQq2gwhz')
# auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

In [ ]:
search_term = "#pti -filter:retweets"

tweets = tw.Cursor(api.search_tweets,
                   q=search_term + 'since:2023-04-30',
                   lang="en").items(1000)

In [ ]:
searched_tweets = [t for t in tweets]

In [ ]:
#Part-3: Creating Dataframe of Tweets

#Cleaning searched tweets and converting into Dataframe
my_list_of_dicts = []
for each_json_tweet in searched_tweets:
    my_list_of_dicts.append(each_json_tweet._json)
    
with open('tweet_pti.txt', 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))
        
my_demo_list = []
with open('tweet_pti.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        text = each_dictionary['text']
        favorite_count = each_dictionary['favorite_count']
        retweet_count = each_dictionary['retweet_count']
        created_at = each_dictionary['created_at']
        my_demo_list.append({'tweet_id': str(tweet_id),
                             'text': str(text),
                             'favorite_count': int(favorite_count),
                             'retweet_count': int(retweet_count),
                             'created_at': created_at,
                            })
        
        tweet_dataset = pd.DataFrame(my_demo_list, columns = 
                                  ['tweet_id', 'text', 
                                   'favorite_count', 'retweet_count', 
                                   'created_at'])
    
#Writing tweet dataset ti csv file for future reference
tweet_dataset.to_csv('tweet_pti.csv',index=False)

In [ ]:
def remove_url(txt):
    #Replace URLs found in a text string with nothing
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [ ]:
import re
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 

In [ ]:
tweet_dataset['text'] = np.vectorize(remove_url)(tweet_dataset['text'])

In [ ]:
tweet_dataset['text'] = np.vectorize(remove_pattern)(tweet_dataset['text'], "@[\w]*")

In [ ]:
tweet_dataset.to_csv('tweet_pti_clean.csv',index=False)

#### b. Perform Sentiment Analysis on the tweet and show total positive and negative sentiment counts.


In [ ]:
def tweet_sentiment(txt):
    analysis = TextBlob(txt)
    if analysis.sentiment[0]>=0:
       res = 'positive'
    else:
       res= 'negative'
    return(res)

In [ ]:
tweet_dataset['sentiment'] = np.vectorize(tweet_sentiment)(tweet_dataset['text'])

In [ ]:
tweet_dataset['sentiment'].value_counts()

#### There are a total of 1000 tweets out of which 898 are positive and 102 are negative

In [ ]:
tweet_dataset.head(20)

In [ ]:
corpus = ' '.join(tweet_dataset.text)
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["T", "J", "D", "S", "M", "H", "et al"])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, max_font_size=50, max_words=100, background_color="white").generate(corpus)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

#### c. Display one Positive and one Negative sentiment and discuss if the algorithm is accurate.


In [ ]:
positive_row = tweet_dataset[tweet_dataset['sentiment'] == 'positive'].sample(n=1)
negative_row = tweet_dataset[tweet_dataset['sentiment'] == 'negative'].sample(n=1)


print("Positive Sentiment:")
print(positive_row)

print("\nNegative Sentiment:")
print(negative_row)


### Analysis Of Algorithm
#### It'll give different values everytime, but the value i've gotten for positive is 172, and 5 for negative, these are indexes. 172 is predicted to positive where as in reality it should have been negative  and number 5 is correct to be negative as it is a negative comment


#### d. Display the tweet with highest retweets and show its sentiment.


In [ ]:
tweet_dataset[tweet_dataset['retweet_count']== tweet_dataset['retweet_count'].max()].sample(n=1)


#### e. Make a timeline visualization plot showing counts of positive and negative sentiments over the period of time.

In [ ]:
grouped = tweet_dataset.groupby(['created_at', 'sentiment']).size().reset_index(name='count')

# Reshape the data for visualization
reshaped = grouped.pivot(index='created_at', columns='sentiment', values='count')

# Plot the timeline visualization
ax = reshaped.plot(kind='area', stacked=True)

# Rotate x-axis labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

plt.xlabel('Time Period')
plt.ylabel('Sentiment Count')
plt.title('Timeline Visualization of Sentiments')
plt.legend()
plt.show()